In [1]:
from bigqueue.client.client import BigqueueClient, BigqueueClientConfig

/home/tianbing_xu/.airconda-environments/production--examples--ai--torchvision--v0.0.3/lib/python3.12/site-packages/bighead/core/util/conversions.py:36: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  PYTHON_SCALAR_TYPES.add(np.bool)


In [2]:
from bigqueue.client.entity import *

In [3]:
config = {
    'bigqueue_service_url': 'https://bigqueue.d.musta.ch',
    'bigqueue_service_version': '0',
    'datastore_type': 'S3',
    'datastore_init_kwargs': {
        'bucket': 'airbnb-bigqueue',
        'base_prefix': 'staging'
    }
}

In [4]:
bigq_client = BigqueueClient(BigqueueClientConfig(**config))

In [5]:
model = bigq_client.upload_model(
    owner="shaowei_su",
    env="devel/shaowei/rl:0.0.4",
    model_file_root='/home/tianbing_xu/repos/GRPO/TinyZero',
    model_file_path='verl/trainer/main_ppo.py',
    name='Demo Ray Job Codebase for MATH RL',
    flavor='ray')

In [ ]:
job = bigq_client.create_ray_job(
    model_id=model['id'],
    name="Test REINFORCE++: base model Qwen2.5-14B-Instruct",
    owner='tianbing_xu',
    team='one_airbnb_team_id_2934',
    execution_config={
        'head': {'cpu': 12, 'memory': '20Gi',  'ephemeral_storage': '50Gi',},
        'worker': {'number': 1, 'gpu': 8, 'accelerator': 'nvidia-tesla-a100xl', 'cpu': 24, 'memory': '400Gi',  'ephemeral_storage': '200Gi',}
    },
    runtime_env={"working_dir": "./", "env_vars": {"WANDB_API_KEY": "09f781a401d2ec1d3ff5f9b550eb1ec0f291fb31"}},
    entrypoint="export WANDB_API_KEY=\"09f781a401d2ec1d3ff5f9b550eb1ec0f291fb31\" && export N_GPUS=8 && export BASE_MODEL=Qwen2.5-14B-Instruct && export DATA_DIR=./data/gsm8k && export ROLLOUT_TP_SIZE=2 && export EXPERIMENT_NAME=reinforce_qwen_14b_inst && export VLLM_ATTENTION_BACKEND=XFORMERS && python3 -m verl.trainer.main_ppo algorithm.adv_estimator=reinforce_plus_plus data.train_files=$DATA_DIR/train.parquet data.val_files=$DATA_DIR/test.parquet data.train_batch_size=32 data.val_batch_size=32 data.max_prompt_length=512 data.max_response_length=1024 data.filter_overlong_prompts=True actor_rollout_ref.model.path=$BASE_MODEL actor_rollout_ref.actor.optim.lr=1e-6 actor_rollout_ref.model.use_remove_padding=True actor_rollout_ref.actor.ppo_mini_batch_size=32 actor_rollout_ref.actor.ppo_micro_batch_size=4 actor_rollout_ref.actor.use_kl_loss=True actor_rollout_ref.actor.kl_loss_coef=0.001 actor_rollout_ref.actor.kl_loss_type=low_var_kl actor_rollout_ref.model.enable_gradient_checkpointing=True actor_rollout_ref.actor.fsdp_config.param_offload=False actor_rollout_ref.actor.fsdp_config.optimizer_offload=False actor_rollout_ref.rollout.log_prob_micro_batch_size=4 actor_rollout_ref.rollout.tensor_model_parallel_size=$ROLLOUT_TP_SIZE actor_rollout_ref.rollout.name=vllm actor_rollout_ref.rollout.gpu_memory_utilization=0.4 actor_rollout_ref.rollout.n=2 actor_rollout_ref.ref.log_prob_micro_batch_size=4 actor_rollout_ref.ref.fsdp_config.param_offload=True algorithm.kl_ctrl.kl_coef=0.001 trainer.logger=\"['console','wandb']\" trainer.project_name=MathRLZero trainer.experiment_name=$EXPERIMENT_NAME trainer.default_hdfs_dir=null trainer.n_gpus_per_node=$N_GPUS trainer.nnodes=1 trainer.save_freq=100 trainer.test_freq=10 trainer.total_epochs=1 trainer.total_training_steps=101",
)

In [9]:
job

{'id': '98356ea2-ac3b-48b7-a3d7-b2df2164efa9'}